In [1]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math
import time
from sklearn.utils import class_weight

# --- 1. Global Configuration ---

# --- Model definitions and their required input sizes ---
MODELS = {
    "MobileNetV2": {"class": tf.keras.applications.MobileNetV2, "input_size": 224},
    "MobileNetV3Large": {"class": tf.keras.applications.MobileNetV3Large, "input_size": 224},
    "EfficientNetB0": {"class": tf.keras.applications.EfficientNetB0, "input_size": 224},
    "EfficientNetB7": {"class": tf.keras.applications.EfficientNetB7, "input_size": 600},
    "InceptionV3": {"class": tf.keras.applications.InceptionV3, "input_size": 299},
    "InceptionResNetV2": {"class": tf.keras.applications.InceptionResNetV2, "input_size": 299},
    "VGG16": {"class": tf.keras.applications.VGG16, "input_size": 224},
    "VGG19": {"class": tf.keras.applications.VGG19, "input_size": 224},
    "ResNet50": {"class": tf.keras.applications.ResNet50, "input_size": 224},
    "ResNet152": {"class": tf.keras.applications.ResNet152, "input_size": 224}
}

# --- Best batch sizes from your initial experiments ---
BEST_BATCH_SIZES = {
    "MobileNet": 8,
    "EfficientNet": 16,
    "Inception": 8,
    "VGG": 16,
    "ResNet": 8
}

RESULTS_DIR = 'results'

# --- 2. The Master Experiment Function ---

def run_experiment(model_choice, training_mode='fine_tuning', batch_size=None, learning_rate=1e-4):
    """
    Runs a complete, configurable training and evaluation experiment.

    Args:
        model_choice (str): Name of the model to run (e.g., "MobileNetV2").
        training_mode (str): One of 'transfer_learning', 'fine_tuning', or 'from_scratch'.
        batch_size (int, optional): Batch size. If None, uses the best default for the model family.
        learning_rate (float, optional): The initial learning rate.
    """
    start_time = time.time()

    # --- Dynamic Setup ---
    model_config = MODELS[model_choice]
    input_size = model_config["input_size"]
    model_family = model_choice.split("V")[0].split("B")[0]
    
    if batch_size is None:
        batch_size = BEST_BATCH_SIZES.get(model_family, 16) # Default to 16 if family not found

    MODEL_NAME = f'{model_choice}_{training_mode}_bs{batch_size}'
    model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
    os.makedirs(model_results_dir, exist_ok=True)

    print("="*80)
    print(f"--- Starting Experiment: {MODEL_NAME} ---")
    print(f"Mode: {training_mode}, Batch Size: {batch_size}, LR: {learning_rate}, Input: {input_size}x{input_size}")
    print("="*80)

    # --- Data Loading (Conditional) ---
    if training_mode == 'from_scratch':
        PROCESSED_DIR = os.path.join('processed_data', f'BrinjalFruitX_{input_size}x{input_size}_balanced_classless')
    elif input_size == 299:
        PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_299x299_balanced_classless')
    elif input_size == 600:
        PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_600x600_balanced_classless')
    else:
        PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_balanced_classless')
    
    print(f"\nLoading data from '{PROCESSED_DIR}'...")
    try:
        X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
        y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
        X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
        y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
        X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
        y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
        with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
            class_names = json.load(f)
        print("Data loaded successfully.")
    except FileNotFoundError:
        print(f"ERROR: Data not found. Please create the required dataset at '{PROCESSED_DIR}'.")
        return

    # --- Model Definition ---
    data_augmentation = models.Sequential([
        layers.RandomFlip("horizontal"), layers.RandomRotation(0.2),
        layers.RandomZoom(0.2), layers.RandomContrast(0.2)], name="data_augmentation")

    base_model_class = model_config["class"]
    
    # --- Training Logic based on Mode ---
    if training_mode == 'from_scratch':
        print("\n--- CONFIGURATION: Training from Scratch ---")
        base_model = base_model_class(input_shape=(input_size, input_size, 3), include_top=False, weights=None)
        base_model.trainable = True
        
        inputs = layers.Input(shape=(input_size, input_size, 3))
        x = data_augmentation(inputs)
        x = base_model(x, training=True)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.5)(x)
        outputs = layers.Dense(len(class_names), activation='softmax')(x)
        model = models.Model(inputs, outputs)

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=20, verbose=1, restore_best_weights=True),
            ModelCheckpoint(filepath=os.path.join(model_results_dir, 'best_model.keras'), monitor='val_loss', save_best_only=True, verbose=1),
            ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=7, verbose=1)
        ]
        
        history = model.fit(X_train, y_train, epochs=100, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=callbacks)

    else: # Handles 'transfer_learning' and 'fine_tuning'
        print(f"\n--- CONFIGURATION: {training_mode.replace('_', ' ').title()} ---")
        base_model = base_model_class(input_shape=(input_size, input_size, 3), include_top=False, weights='imagenet')
        base_model.trainable = False
        
        inputs = layers.Input(shape=(input_size, input_size, 3))
        x = data_augmentation(inputs)
        x = base_model(x, training=False)
        x = layers.GlobalAveragePooling2D()(x)
        x = layers.Dropout(0.5)(x)
        outputs = layers.Dense(len(class_names), activation='softmax')(x)
        model = models.Model(inputs, outputs)

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)])
        
        if training_mode == 'fine_tuning':
            print("\n--- STAGE 2: Fine-Tuning ---")
            class_labels = np.unique(y_train)
            weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=y_train)
            class_weights_dict = dict(zip(class_labels, weights))
            print("Applying Class Weights:", class_weights_dict)

            base_model.trainable = True
            fine_tune_at = math.ceil(len(base_model.layers) * 0.5)
            for layer in base_model.layers[:fine_tune_at]:
                layer.trainable = False
            
            model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            
            callbacks_finetune = [
                EarlyStopping(monitor='val_loss', patience=15, verbose=1),
                ModelCheckpoint(filepath=os.path.join(model_results_dir, 'best_model.keras'), monitor='val_loss', save_best_only=True, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1)
            ]
            
            total_epochs = history.epoch[-1] + 1 + 50
            history_fine_tune = model.fit(X_train, y_train, epochs=total_epochs, initial_epoch=history.epoch[-1] + 1, batch_size=batch_size, validation_data=(X_val, y_val), class_weight=class_weights_dict, callbacks=callbacks_finetune)
            # Combine history objects for plotting
            history.history['accuracy'].extend(history_fine_tune.history['accuracy'])
            history.history['val_accuracy'].extend(history_fine_tune.history['val_accuracy'])
            history.history['loss'].extend(history_fine_tune.history['loss'])
            history.history['val_loss'].extend(history_fine_tune.history['val_loss'])
            
            model.load_model(os.path.join(model_results_dir, 'best_model.keras'))

    # --- ADDED: Visualization Section ---
    print("\nGenerating and saving performance plot...")
    plt.figure(figsize=(12, 8))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{MODEL_NAME}: Model Accuracy & Loss')
    plt.ylabel('Accuracy & Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.savefig(os.path.join(model_results_dir, 'performance_plot.png'))
    plt.close() # Close the plot to free up memory

    # --- Evaluation and Saving ---
    end_time = time.time()
    training_duration = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))
    
    print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
    y_pred_test_probs = model.predict(X_test)
    y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
    report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
    print("\nTest Set Classification Report:\n")
    print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

    cm = confusion_matrix(y_test, y_pred_test_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{MODEL_NAME} Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
    plt.close() # Close the plot
    
    print("\nUpdating summary results file...")
    summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    summary_data = {
        'model_name': MODEL_NAME,
        'test_accuracy': f"{test_accuracy:.4f}",
        'test_loss': f"{test_loss:.4f}",
        'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
        'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}",
        'training_time': training_duration
    }
    new_results_df = pd.DataFrame([summary_data])
    if os.path.exists(summary_file):
        summary_df = pd.read_csv(summary_file)
        if MODEL_NAME in summary_df['model_name'].values:
            summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
        else:
            summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
        summary_df.to_csv(summary_file, index=False)
    else:
        new_results_df.to_csv(summary_file, index=False)

    print(f"\n--- Experiment for {MODEL_NAME} is complete. Total time: {training_duration} ---")

In [ ]:
# --- 3. Main Execution Block ---

if __name__ == "__main__":
    # --- Example of how to run experiments ---
    
    # --- Run the best fine-tuning experiment for a specific model ---
    run_experiment(
        model_choice="MobileNetV2",
        training_mode='fine_tuning'
    )
    
    # --- Run a simple transfer learning experiment ---
    # run_experiment(
    #     model_choice="ResNet50",
    #     training_mode='transfer_learning',
    #     batch_size=32,
    #     learning_rate=1e-3
    # )

    # --- Run a 'from scratch' experiment ---
    # NOTE: This requires a balanced dataset to be created first.
    # run_experiment(
    #     model_choice="EfficientNetB0",
    #     training_mode='from_scratch'
    # )

In [1]:
import os
import numpy as np
import json
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import math
import time
from sklearn.utils import class_weight

# --- 1. Global Configuration ---

# --- Model definitions and their required input sizes ---
MODELS = {
    "MobileNetV2": {"class": tf.keras.applications.MobileNetV2, "input_size": 224},
    "MobileNetV3Large": {"class": tf.keras.applications.MobileNetV3Large, "input_size": 224},
    "EfficientNetB0": {"class": tf.keras.applications.EfficientNetB0, "input_size": 224},
    "EfficientNetB7": {"class": tf.keras.applications.EfficientNetB7, "input_size": 600},
    "InceptionV3": {"class": tf.keras.applications.InceptionV3, "input_size": 299},
    "InceptionResNetV2": {"class": tf.keras.applications.InceptionResNetV2, "input_size": 299},
    "VGG16": {"class": tf.keras.applications.VGG16, "input_size": 224},
    "VGG19": {"class": tf.keras.applications.VGG19, "input_size": 224},
    "ResNet50": {"class": tf.keras.applications.ResNet50, "input_size": 224},
    "ResNet152": {"class": tf.keras.applications.ResNet152, "input_size": 224}
}

# --- Best batch sizes from your initial experiments ---
BEST_BATCH_SIZES = {
    "MobileNet": 8,
    "EfficientNet": 16,
    "Inception": 8,
    "VGG": 16,
    "ResNet": 8
}

RESULTS_DIR = 'results'

# --- 2. The Master Experiment Function ---

def run_experiment(model_choice, training_mode='full_monty', batch_size=None, learning_rate=1e-4):
    """
    Runs a complete, configurable training and evaluation experiment.

    Args:
        model_choice (str): Name of the model to run (e.g., "MobileNetV2").
        training_mode (str): One of 'transfer_learning', 'fine_tuning', 'from_scratch', or 'full_monty'.
        batch_size (int, optional): Batch size. If None, uses the best default for the model family.
        learning_rate (float, optional): The initial learning rate.
    """
    start_time = time.time()

    # --- Dynamic Setup ---
    model_config = MODELS[model_choice]
    input_size = model_config["input_size"]
    model_family = model_choice.split("V")[0].split("B")[0]
    
    if batch_size is None:
        batch_size = BEST_BATCH_SIZES.get(model_family, 16)

    MODEL_NAME = f'{model_choice}_{training_mode}_bs{batch_size}'
    model_results_dir = os.path.join(RESULTS_DIR, MODEL_NAME)
    os.makedirs(model_results_dir, exist_ok=True)

    print("="*80)
    print(f"--- Starting Experiment: {MODEL_NAME} ---")
    print(f"Mode: {training_mode}, Batch Size: {batch_size}, LR: {learning_rate}, Input: {input_size}x{input_size}")
    print("="*80)

    # --- Data Loading (Conditional) ---
    use_balanced_data = training_mode in ['from_scratch', 'full_monty']
    
    if use_balanced_data:
        PROCESSED_DIR = os.path.join('processed_data', f'BrinjalFruitX_{input_size}x{input_size}_balanced_classless')
    elif input_size == 299:
        PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_299x299_balanced_classless')
    elif input_size == 600:
        PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_600x600_balanced_classless')
    else:
        PROCESSED_DIR = os.path.join('processed_data', 'BrinjalFruitX_balanced_classless')
    
    print(f"\nLoading data from '{PROCESSED_DIR}'...")
    try:
        X_train = np.load(os.path.join(PROCESSED_DIR, 'X_train.npy'))
        y_train = np.load(os.path.join(PROCESSED_DIR, 'y_train.npy'))
        X_val = np.load(os.path.join(PROCESSED_DIR, 'X_val.npy'))
        y_val = np.load(os.path.join(PROCESSED_DIR, 'y_val.npy'))
        X_test = np.load(os.path.join(PROCESSED_DIR, 'X_test.npy'))
        y_test = np.load(os.path.join(PROCESSED_DIR, 'y_test.npy'))
        with open(os.path.join(PROCESSED_DIR, 'class_names.json'), 'r') as f:
            class_names = json.load(f)
        print("Data loaded successfully.")
    except FileNotFoundError:
        print(f"ERROR: Data not found. Please create the required dataset at '{PROCESSED_DIR}'.")
        return

    # --- Model Definition ---
    data_augmentation = models.Sequential([
        layers.RandomFlip("horizontal"), layers.RandomRotation(0.2),
        layers.RandomZoom(0.2), layers.RandomContrast(0.2)], name="data_augmentation")

    base_model_class = model_config["class"]
    
    # --- Training Logic based on Mode ---
    use_transfer_weights = training_mode != 'from_scratch'
    
    print(f"\n--- CONFIGURATION: {training_mode.replace('_', ' ').title()} ---")
    base_model = base_model_class(
        input_shape=(input_size, input_size, 3), 
        include_top=False, 
        weights='imagenet' if use_transfer_weights else None
    )
    base_model.trainable = not use_transfer_weights

    inputs = layers.Input(shape=(input_size, input_size, 3))
    x = data_augmentation(inputs)
    x = base_model(x, training=(not use_transfer_weights))
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(len(class_names), activation='softmax')(x)
    model = models.Model(inputs, outputs)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
        ModelCheckpoint(filepath=os.path.join(model_results_dir, 'best_model.keras'), monitor='val_loss', save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1)
    ]
    
    fit_kwargs = {'class_weight': None}
    if training_mode == 'fine_tuning': # Only use class_weight for this specific mode
        class_labels = np.unique(y_train)
        weights = class_weight.compute_class_weight('balanced', classes=class_labels, y=y_train)
        fit_kwargs['class_weight'] = dict(zip(class_labels, weights))
        print("Applying Class Weights for training on imbalanced data.")

    history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=callbacks, **fit_kwargs)
    
    is_two_stage = training_mode in ['fine_tuning', 'full_monty']
    if is_two_stage:
        print("\n--- STAGE 2: Fine-Tuning ---")
        base_model.trainable = True
        fine_tune_at = math.ceil(len(base_model.layers) * 0.5)
        for layer in base_model.layers[:fine_tune_at]:
            layer.trainable = False
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        total_epochs = history.epoch[-1] + 1 + 50
        history_fine_tune = model.fit(X_train, y_train, epochs=total_epochs, initial_epoch=history.epoch[-1] + 1, batch_size=batch_size, validation_data=(X_val, y_val), callbacks=callbacks, **fit_kwargs)
        
        # Combine history objects for plotting
        history.history['accuracy'].extend(history_fine_tune.history['accuracy'])
        history.history['val_accuracy'].extend(history_fine_tune.history['val_accuracy'])
        history.history['loss'].extend(history_fine_tune.history['loss'])
        history.history['val_loss'].extend(history_fine_tune.history['val_loss'])
        
        model = tf.keras.models.load_model(os.path.join(model_results_dir, 'best_model.keras'))


    # --- ADDED: Visualization Section ---
    print("\nGenerating and saving performance plot...")
    plt.figure(figsize=(12, 8))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{MODEL_NAME}: Model Accuracy & Loss')
    plt.ylabel('Accuracy & Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='upper right')
    plt.savefig(os.path.join(model_results_dir, 'performance_plot.png'))
    plt.close() # Close the plot to free up memory

    # --- Evaluation and Saving ---
    end_time = time.time()
    training_duration = time.strftime("%H:%M:%S", time.gmtime(end_time - start_time))
    
    print(f"\n--- {MODEL_NAME} Final Test Set Evaluation ---")
    y_pred_test_probs = model.predict(X_test)
    y_pred_test_classes = np.argmax(y_pred_test_probs, axis=1)
    report_dict = classification_report(y_test, y_pred_test_classes, target_names=class_names, output_dict=True)
    print("\nTest Set Classification Report:\n")
    print(classification_report(y_test, y_pred_test_classes, target_names=class_names))

    cm = confusion_matrix(y_test, y_pred_test_classes)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'{MODEL_NAME} Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(os.path.join(model_results_dir, 'confusion_matrix.png'))
    plt.close() # Close the plot
    
    print("\nUpdating summary results file...")
    summary_file = os.path.join(RESULTS_DIR, 'summary_results.csv')
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    summary_data = {
        'model_name': MODEL_NAME,
        'test_accuracy': f"{test_accuracy:.4f}",
        'test_loss': f"{test_loss:.4f}",
        'macro_avg_f1-score': f"{report_dict['macro avg']['f1-score']:.4f}",
        'weighted_avg_f1-score': f"{report_dict['weighted avg']['f1-score']:.4f}",
        'training_time': training_duration
    }
    new_results_df = pd.DataFrame([summary_data])
    if os.path.exists(summary_file):
        summary_df = pd.read_csv(summary_file)
        if MODEL_NAME in summary_df['model_name'].values:
            summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
        else:
            summary_df = pd.concat([summary_df, new_results_df], ignore_index=True)
        summary_df.to_csv(summary_file, index=False)
    else:
        new_results_df.to_csv(summary_file, index=False)

    print(f"\n--- Experiment for {MODEL_NAME} is complete. Total time: {training_duration} ---")

In [2]:
run_experiment(
    model_choice="MobileNetV2",
    training_mode='from_scratch'
)

--- Starting Experiment: MobileNetV2_from_scratch_bs8 ---
Mode: from_scratch, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.6481 - loss: 0.7053
Epoch 1: val_loss improved from inf to 0.68157, saving model to results\MobileNetV2_from_scratch_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 55s 269ms/step - accuracy: 0.6481 - loss: 0.7051 - val_accuracy: 0.5887 - val_loss: 0.6816 - learning_rate: 1.0000e-04
Epoch 2/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.6902 - loss: 0.6368
Epoch 2: val_loss improved from 0.68157 to 0.67767, saving model to results\MobileNetV2_from_scratch_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 33s 259ms/step - accuracy: 0.6903 - loss: 0.6363 - val_accuracy: 0.5887 - val_loss: 0.6777 - learning_rate: 1.0000e-04
Epoch 3/100
12

c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf


--- Experiment for MobileNetV2_from_scratch_bs8 is complete. Total time: 00:12:21 ---


C:\Users\dr-basab\AppData\Local\Temp\ipykernel_15668\1544972724.py:225: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.5847' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
C:\Users\dr-basab\AppData\Local\Temp\ipykernel_15668\1544972724.py:225: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.6789' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  summary_df.loc[summary_df['model_name'] == MODEL_NAME] = new_results_df.iloc[0].values
C:\Users\dr-basab\AppData\Local\Temp\ipykernel_15668\1544972724.py:225: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.3690' has dtype incompati

In [3]:
run_experiment(
    model_choice="MobileNetV2",
    training_mode='transfer_learning'
)

--- Starting Experiment: MobileNetV2_transfer_learning_bs8 ---
Mode: transfer_learning, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 13s 77ms/step - accuracy: 0.5635 - loss: 0.8881 - val_accuracy: 0.6048 - val_loss: 0.6180
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.6535 - loss: 0.6961 - val_accuracy: 0.7016 - val_loss: 0.5421
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - accuracy: 0.7671 - loss: 0.5229 - val_accuracy: 0.7661 - val_loss: 0.4633
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 67ms/step - accuracy: 0.7502 - loss: 0.5279 - val_accuracy: 0.7742 - val_loss: 0.4620
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 8s 66ms/step - accuracy: 0.7980 - loss: 0.4475 - val_accuracy: 0.7903 - val_loss: 0.4170
Epoch 6/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.8398 - loss: 0.38

In [24]:
run_experiment(
    model_choice="MobileNetV2",
    training_mode='fine_tune'
)

--- Starting Experiment: MobileNetV2_fine_tune_bs8 ---
Mode: fine_tune, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 45s 325ms/step - accuracy: 0.5145 - loss: 1.0096 - val_accuracy: 0.6532 - val_loss: 0.6676
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 40s 316ms/step - accuracy: 0.6009 - loss: 0.7333 - val_accuracy: 0.7177 - val_loss: 0.5552
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 41s 324ms/step - accuracy: 0.6773 - loss: 0.6510 - val_accuracy: 0.7742 - val_loss: 0.4820
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 40s 316ms/step - accuracy: 0.7715 - loss: 0.4845 - val_accuracy: 0.7903 - val_loss: 0.4552
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 41s 323ms/step - accuracy: 0.7746 - loss: 0.4919 - val_accuracy: 0.8065 - val_loss: 0.4289
Epoch 6/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 40s 314ms/step - accuracy: 0.8192 - loss: 0.3809 - val_accu

In [2]:
run_experiment(
    model_choice="MobileNetV2",
    training_mode='full_monty'
)

--- Starting Experiment: MobileNetV2_full_monty_bs8 ---
Mode: full_monty, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5660 - loss: 0.8670
Epoch 1: val_loss improved from inf to 0.65366, saving model to results\MobileNetV2_full_monty_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 14s 81ms/step - accuracy: 0.5662 - loss: 0.8667 - val_accuracy: 0.6452 - val_loss: 0.6537 - learning_rate: 1.0000e-04
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.6653 - loss: 0.6551
Epoch 2: val_loss improved from 0.65366 to 0.53037, saving model to results\MobileNetV2_full_monty_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.6655 - loss: 0.6549 - val_accuracy: 0.7097 - val_loss: 0.5304 - learning_rate: 1.0000e-04
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 

In [4]:
run_experiment(
    model_choice="MobileNetV3Large",
    training_mode='from_scratch'
)

--- Starting Experiment: MobileNetV3Large_from_scratch_bs8 ---
Mode: from_scratch, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step - accuracy: 0.5752 - loss: 0.6859
Epoch 1: val_loss improved from inf to 0.69107, saving model to results\MobileNetV3Large_from_scratch_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 56s 262ms/step - accuracy: 0.5756 - loss: 0.6855 - val_accuracy: 0.5887 - val_loss: 0.6911 - learning_rate: 1.0000e-04
Epoch 2/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.7461 - loss: 0.5484
Epoch 2: val_loss did not improve from 0.69107
127/127 ━━━━━━━━━━━━━━━━━━━━ 30s 235ms/step - accuracy: 0.7460 - loss: 0.5484 - val_accuracy: 0.5887 - val_loss: 0.6912 - learning_rate: 1.0000e-04
Epoch 3/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step - accuracy: 0.7397 - los

c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf


--- Experiment for MobileNetV3Large_from_scratch_bs8 is complete. Total time: 00:26:39 ---


In [5]:
run_experiment(
    model_choice="MobileNetV3Large",
    training_mode='transfer_learning'
)

--- Starting Experiment: MobileNetV3Large_transfer_learning_bs8 ---
Mode: transfer_learning, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 15s 81ms/step - accuracy: 0.5014 - loss: 0.7969 - val_accuracy: 0.4113 - val_loss: 0.7151
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - accuracy: 0.4428 - loss: 0.7518 - val_accuracy: 0.4032 - val_loss: 0.6989
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 69ms/step - accuracy: 0.4853 - loss: 0.7348 - val_accuracy: 0.5887 - val_loss: 0.6928
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.5186 - loss: 0.7292 - val_accuracy: 0.5806 - val_loss: 0.6891
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 70ms/step - accuracy: 0.4836 - loss: 0.7200 - val_accuracy: 0.5968 - val_loss: 0.6858
Epoch 6/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 71ms/step - accuracy: 0.5283 - loss:

In [14]:
run_experiment(
    model_choice="MobileNetV3Large",
    training_mode='fine_tune'
)

--- Starting Experiment: MobileNetV3Large_fine_tune_bs8 ---
Mode: fine_tune, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 31s 210ms/step - accuracy: 0.4894 - loss: 0.7667 - val_accuracy: 0.5968 - val_loss: 0.6880
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 24s 192ms/step - accuracy: 0.5305 - loss: 0.7446 - val_accuracy: 0.5161 - val_loss: 0.6915
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 24s 192ms/step - accuracy: 0.4719 - loss: 0.7404 - val_accuracy: 0.6774 - val_loss: 0.6891
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 25s 195ms/step - accuracy: 0.5166 - loss: 0.7126 - val_accuracy: 0.4516 - val_loss: 0.6913
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 25s 194ms/step - accuracy: 0.4695 - loss: 0.7464 - val_accuracy: 0.4355 - val_loss: 0.6905
Epoch 6/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 25s 196ms/step - accuracy: 0.5198 - loss: 0.7306 - val

In [3]:
run_experiment(
    model_choice="MobileNetV3Large",
    training_mode='full_monty'
)

--- Starting Experiment: MobileNetV3Large_full_monty_bs8 ---
Mode: full_monty, Batch Size: 8, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - accuracy: 0.4546 - loss: 0.8186
Epoch 1: val_loss improved from inf to 0.69612, saving model to results\MobileNetV3Large_full_monty_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 16s 88ms/step - accuracy: 0.4548 - loss: 0.8183 - val_accuracy: 0.3710 - val_loss: 0.6961 - learning_rate: 1.0000e-04
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.5259 - loss: 0.7368
Epoch 2: val_loss improved from 0.69612 to 0.69518, saving model to results\MobileNetV3Large_full_monty_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 9s 67ms/step - accuracy: 0.5259 - loss: 0.7367 - val_accuracy: 0.3952 - val_loss: 0.6952 - learning_rate: 1.0000e-04
Epoch 3/50
127/127 ━━━━━━

In [6]:
run_experiment(
    model_choice="EfficientNetB0",
    training_mode='from_scratch'
)

--- Starting Experiment: EfficientNetB0_from_scratch_bs16 ---
Mode: from_scratch, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 770ms/step - accuracy: 0.5615 - loss: 0.7385
Epoch 1: val_loss improved from inf to 0.68612, saving model to results\EfficientNetB0_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 84s 833ms/step - accuracy: 0.5617 - loss: 0.7381 - val_accuracy: 0.5887 - val_loss: 0.6861 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 715ms/step - accuracy: 0.6320 - loss: 0.6685
Epoch 2: val_loss did not improve from 0.68612
64/64 ━━━━━━━━━━━━━━━━━━━━ 47s 734ms/step - accuracy: 0.6320 - loss: 0.6685 - val_accuracy: 0.5887 - val_loss: 0.6903 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 708ms/step - accuracy: 0.7206 - loss: 0.5800
E

In [7]:
run_experiment(
    model_choice="EfficientNetB0",
    training_mode='transfer_learning'
)

--- Starting Experiment: EfficientNetB0_transfer_learning_bs16 ---
Mode: transfer_learning, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 23s 235ms/step - accuracy: 0.4756 - loss: 0.7283 - val_accuracy: 0.4113 - val_loss: 0.6957
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step - accuracy: 0.4757 - loss: 0.7212 - val_accuracy: 0.5887 - val_loss: 0.6886
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step - accuracy: 0.4875 - loss: 0.7132 - val_accuracy: 0.4113 - val_loss: 0.6984
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step - accuracy: 0.5233 - loss: 0.7050 - val_accuracy: 0.4113 - val_loss: 0.6971
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step - accuracy: 0.5130 - loss: 0.7104 - val_accuracy: 0.5887 - val_loss: 0.6924
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 187ms/step - accuracy: 0.4559 - loss: 

c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf


--- Experiment for EfficientNetB0_transfer_learning_bs16 is complete. Total time: 00:08:58 ---


In [15]:
run_experiment(
    model_choice="EfficientNetB0",
    training_mode='fine_tune'
)

--- Starting Experiment: EfficientNetB0_fine_tune_bs16 ---
Mode: fine_tune, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 46s 559ms/step - accuracy: 0.5173 - loss: 0.7050 - val_accuracy: 0.4113 - val_loss: 0.6970
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 33s 516ms/step - accuracy: 0.4982 - loss: 0.7113 - val_accuracy: 0.4113 - val_loss: 0.6973
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 33s 516ms/step - accuracy: 0.5093 - loss: 0.7061 - val_accuracy: 0.3952 - val_loss: 0.6932
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 33s 520ms/step - accuracy: 0.4843 - loss: 0.7246 - val_accuracy: 0.4113 - val_loss: 0.6998
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 33s 521ms/step - accuracy: 0.4964 - loss: 0.7208 - val_accuracy: 0.5887 - val_loss: 0.6823
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 33s 521ms/step - accuracy: 0.5094 - loss: 0.7129 - val_accuracy: 0

c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\dr-basab\Desktop\Research Projects\ADLMODCT\venv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf


--- Experiment for EfficientNetB0_fine_tune_bs16 is complete. Total time: 00:11:46 ---


In [4]:
run_experiment(
    model_choice="EfficientNetB0",
    training_mode='full_monty'
)

--- Starting Experiment: EfficientNetB0_full_monty_bs16 ---
Mode: full_monty, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - accuracy: 0.4900 - loss: 0.7202
Epoch 1: val_loss improved from inf to 0.69053, saving model to results\EfficientNetB0_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 20s 201ms/step - accuracy: 0.4900 - loss: 0.7202 - val_accuracy: 0.5887 - val_loss: 0.6905 - learning_rate: 1.0000e-04
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.4969 - loss: 0.7101
Epoch 2: val_loss did not improve from 0.69053
64/64 ━━━━━━━━━━━━━━━━━━━━ 9s 144ms/step - accuracy: 0.4969 - loss: 0.7101 - val_accuracy: 0.4113 - val_loss: 0.6986 - learning_rate: 1.0000e-04
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.4559 - loss: 0.7303
Epoch 3: val_loss did 

In [ ]:
run_experiment(
    model_choice="EfficientNetB7",
    training_mode='from_scratch'
)

--- Starting Experiment: EfficientNetB7_from_scratch_bs16 ---
Mode: from_scratch, Batch Size: 16, LR: 0.0001, Input: 600x600

Loading data from 'processed_data\BrinjalFruitX_600x600_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 132s/step - accuracy: 0.5690 - loss: 0.7128  
Epoch 1: val_loss improved from inf to 0.69374, saving model to results\EfficientNetB7_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 8689s 134s/step - accuracy: 0.5693 - loss: 0.7131 - val_accuracy: 0.4113 - val_loss: 0.6937 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 134s/step - accuracy: 0.5878 - loss: 0.6914  
Epoch 2: val_loss did not improve from 0.69374
64/64 ━━━━━━━━━━━━━━━━━━━━ 8675s 135s/step - accuracy: 0.5877 - loss: 0.6913 - val_accuracy: 0.5887 - val_loss: 0.7652 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 132s/step - accuracy: 0.6627 - loss: 0.648

In [ ]:
run_experiment(
    model_choice="EfficientNetB7",
    training_mode='transfer_learning'
)

In [2]:
run_experiment(
    model_choice="InceptionV3",
    training_mode='from_scratch'
)

--- Starting Experiment: InceptionV3_from_scratch_bs8 ---
Mode: from_scratch, Batch Size: 8, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 565ms/step - accuracy: 0.5876 - loss: 0.7744
Epoch 1: val_loss improved from inf to 0.98280, saving model to results\InceptionV3_from_scratch_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 110s 620ms/step - accuracy: 0.5885 - loss: 0.7733 - val_accuracy: 0.4113 - val_loss: 0.9828 - learning_rate: 1.0000e-04
Epoch 2/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 541ms/step - accuracy: 0.7727 - loss: 0.5022
Epoch 2: val_loss did not improve from 0.98280
127/127 ━━━━━━━━━━━━━━━━━━━━ 71s 558ms/step - accuracy: 0.7727 - loss: 0.5023 - val_accuracy: 0.5887 - val_loss: 1.0540 - learning_rate: 1.0000e-04
Epoch 3/100
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 543ms/step - accuracy: 0.8166 - loss: 0.4423

In [3]:
run_experiment(
    model_choice="InceptionV3",
    training_mode='transfer_learning'
)

--- Starting Experiment: InceptionV3_transfer_learning_bs8 ---
Mode: transfer_learning, Batch Size: 8, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 32s 207ms/step - accuracy: 0.5543 - loss: 0.7270 - val_accuracy: 0.7016 - val_loss: 0.5914
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step - accuracy: 0.6875 - loss: 0.6038 - val_accuracy: 0.7581 - val_loss: 0.5282
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 183ms/step - accuracy: 0.7313 - loss: 0.5545 - val_accuracy: 0.8065 - val_loss: 0.4898
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 184ms/step - accuracy: 0.7630 - loss: 0.5194 - val_accuracy: 0.8226 - val_loss: 0.4646
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 184ms/step - accuracy: 0.7980 - loss: 0.4455 - val_accuracy: 0.8548 - val_loss: 0.4259
Epoch 6/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 23s 184ms/step - accuracy: 

In [16]:
run_experiment(
    model_choice="InceptionV3",
    training_mode='fine_tune'
)

--- Starting Experiment: InceptionV3_fine_tune_bs8 ---
Mode: fine_tune, Batch Size: 8, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 72s 527ms/step - accuracy: 0.5008 - loss: 0.7881 - val_accuracy: 0.7258 - val_loss: 0.5911
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 65s 508ms/step - accuracy: 0.6653 - loss: 0.6136 - val_accuracy: 0.6774 - val_loss: 0.6096
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 65s 510ms/step - accuracy: 0.6946 - loss: 0.5858 - val_accuracy: 0.8306 - val_loss: 0.5115
Epoch 4/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 65s 510ms/step - accuracy: 0.7561 - loss: 0.5214 - val_accuracy: 0.8306 - val_loss: 0.4603
Epoch 5/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 65s 509ms/step - accuracy: 0.7770 - loss: 0.4743 - val_accuracy: 0.6935 - val_loss: 0.5402
Epoch 6/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 65s 510ms/step - accuracy: 0.7728 - loss: 0.4777 - 

In [5]:
run_experiment(
    model_choice="InceptionV3",
    training_mode='full_monty'
)

--- Starting Experiment: InceptionV3_full_monty_bs8 ---
Mode: full_monty, Batch Size: 8, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.5123 - loss: 0.7975
Epoch 1: val_loss improved from inf to 0.67467, saving model to results\InceptionV3_full_monty_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 28s 179ms/step - accuracy: 0.5124 - loss: 0.7974 - val_accuracy: 0.5968 - val_loss: 0.6747 - learning_rate: 1.0000e-04
Epoch 2/50
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.6185 - loss: 0.7042
Epoch 2: val_loss improved from 0.67467 to 0.61797, saving model to results\InceptionV3_full_monty_bs8\best_model.keras
127/127 ━━━━━━━━━━━━━━━━━━━━ 20s 161ms/step - accuracy: 0.6185 - loss: 0.7042 - val_accuracy: 0.6774 - val_loss: 0.6180 - learning_rate: 1.0000e-04
Epoch 3/50
127/127 ━━━━━━━━━━━━━━━━

In [4]:
run_experiment(
    model_choice="InceptionResNetV2",
    training_mode='from_scratch'
)

--- Starting Experiment: InceptionResNetV2_from_scratch_bs16 ---
Mode: from_scratch, Batch Size: 16, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---

Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.7541 - loss: 0.5366
Epoch 1: val_loss improved from inf to 1.01766, saving model to results\InceptionResNetV2_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 287s 3s/step - accuracy: 0.7549 - loss: 0.5355 - val_accuracy: 0.5887 - val_loss: 1.0177 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8539 - loss: 0.3881
Epoch 2: val_loss did not improve from 1.01766
64/64 ━━━━━━━━━━━━━━━━━━━━ 198s 3s/step - accuracy: 0.8538 - loss: 0.3881 - val_accuracy: 0.5887 - val_loss: 2.4534 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8755 - loss: 0.3640
Epoch 3

In [5]:
run_experiment(
    model_choice="InceptionResNetV2",
    training_mode='transfer_learning'
)

--- Starting Experiment: InceptionResNetV2_transfer_learning_bs16 ---
Mode: transfer_learning, Batch Size: 16, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 98s 1s/step - accuracy: 0.4940 - loss: 0.8416 - val_accuracy: 0.4919 - val_loss: 0.7430
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.5459 - loss: 0.7733 - val_accuracy: 0.5565 - val_loss: 0.7124
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.5993 - loss: 0.7218 - val_accuracy: 0.5645 - val_loss: 0.7117
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.6530 - loss: 0.6650 - val_accuracy: 0.6371 - val_loss: 0.6499
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6572 - loss: 0.6378 - val_accuracy: 0.6694 - val_loss: 0.6273
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.7173 - loss: 0.5912 

In [17]:
run_experiment(
    model_choice="InceptionResNetV2",
    training_mode='fine_tune'
)

--- Starting Experiment: InceptionResNetV2_fine_tune_bs16 ---
Mode: fine_tune, Batch Size: 16, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5000 - loss: 0.8546 - val_accuracy: 0.6048 - val_loss: 0.6312
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.5719 - loss: 0.7529 - val_accuracy: 0.6532 - val_loss: 0.6040
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.6329 - loss: 0.6738 - val_accuracy: 0.6855 - val_loss: 0.5842
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.6450 - loss: 0.6550 - val_accuracy: 0.7177 - val_loss: 0.5587
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.6818 - loss: 0.6082 - val_accuracy: 0.7016 - val_loss: 0.5331
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 117s 2s/step - accuracy: 0.6895 - loss: 0.5832 - val_accuracy: 0.

In [6]:
run_experiment(
    model_choice="InceptionResNetV2",
    training_mode='full_monty'
)

--- Starting Experiment: InceptionResNetV2_full_monty_bs16 ---
Mode: full_monty, Batch Size: 16, LR: 0.0001, Input: 299x299

Loading data from 'processed_data\BrinjalFruitX_299x299_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---

Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 832ms/step - accuracy: 0.5214 - loss: 0.7977
Epoch 1: val_loss improved from inf to 0.65781, saving model to results\InceptionResNetV2_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.5214 - loss: 0.7975 - val_accuracy: 0.5968 - val_loss: 0.6578 - learning_rate: 1.0000e-04
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 811ms/step - accuracy: 0.5607 - loss: 0.7417
Epoch 2: val_loss improved from 0.65781 to 0.62041, saving model to results\InceptionResNetV2_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 61s 947ms/step - accuracy: 0.5609 - loss: 0.7414 - val_accuracy: 0.6532 - val_loss: 0.6204 - learning_rate: 1.0000e-04
Epoch 3/50
64/64 ━━━━━━

In [6]:
run_experiment(
    model_choice="VGG16",
    training_mode='from_scratch'
)

--- Starting Experiment: VGG16_from_scratch_bs16 ---
Mode: from_scratch, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4953 - loss: 0.6937
Epoch 1: val_loss improved from inf to 0.67520, saving model to results\VGG16_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 101s 2s/step - accuracy: 0.4957 - loss: 0.6938 - val_accuracy: 0.5968 - val_loss: 0.6752 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5736 - loss: 0.6836
Epoch 2: val_loss improved from 0.67520 to 0.49016, saving model to results\VGG16_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.5741 - loss: 0.6832 - val_accuracy: 0.7500 - val_loss: 0.4902 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/st

In [7]:
run_experiment(
    model_choice="VGG16",
    training_mode='transfer_learning'
)

--- Starting Experiment: VGG16_transfer_learning_bs16 ---
Mode: transfer_learning, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 435ms/step - accuracy: 0.4924 - loss: 1.0555 - val_accuracy: 0.4113 - val_loss: 0.8182
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 443ms/step - accuracy: 0.5290 - loss: 0.9395 - val_accuracy: 0.3871 - val_loss: 0.7613
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 440ms/step - accuracy: 0.4750 - loss: 1.0172 - val_accuracy: 0.4516 - val_loss: 0.7235
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 448ms/step - accuracy: 0.4841 - loss: 0.9584 - val_accuracy: 0.4839 - val_loss: 0.7008
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 441ms/step - accuracy: 0.4888 - loss: 0.8865 - val_accuracy: 0.5645 - val_loss: 0.6854
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 448ms/step - accuracy: 0.5414 - loss: 0.8160 - 

In [18]:
run_experiment(
    model_choice="VGG16",
    training_mode='fine_tune'
)

--- Starting Experiment: VGG16_fine_tune_bs16 ---
Mode: fine_tune, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 458ms/step - accuracy: 0.4823 - loss: 0.8056 - val_accuracy: 0.4274 - val_loss: 0.7429
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 464ms/step - accuracy: 0.4819 - loss: 0.7636 - val_accuracy: 0.4597 - val_loss: 0.7070
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 470ms/step - accuracy: 0.4947 - loss: 0.7420 - val_accuracy: 0.5403 - val_loss: 0.6866
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 463ms/step - accuracy: 0.4888 - loss: 0.7373 - val_accuracy: 0.6129 - val_loss: 0.6689
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 465ms/step - accuracy: 0.5132 - loss: 0.7132 - val_accuracy: 0.6532 - val_loss: 0.6603
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 463ms/step - accuracy: 0.4928 - loss: 0.7093 - val_accuracy: 0.7097 - v

In [7]:
run_experiment(
    model_choice="VGG16",
    training_mode='full_monty'
)

--- Starting Experiment: VGG16_full_monty_bs16 ---
Mode: full_monty, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 390ms/step - accuracy: 0.5181 - loss: 0.9954
Epoch 1: val_loss improved from inf to 0.69489, saving model to results\VGG16_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 30s 446ms/step - accuracy: 0.5178 - loss: 0.9951 - val_accuracy: 0.5887 - val_loss: 0.6949 - learning_rate: 1.0000e-04
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 406ms/step - accuracy: 0.4656 - loss: 0.9078
Epoch 2: val_loss improved from 0.69489 to 0.67227, saving model to results\VGG16_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 460ms/step - accuracy: 0.4660 - loss: 0.9071 - val_accuracy: 0.5806 - val_loss: 0.6723 - learning_rate: 1.0000e-04
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 405ms/step - acc

In [8]:
run_experiment(
    model_choice="VGG19",
    training_mode='from_scratch'
)

--- Starting Experiment: VGG19_from_scratch_bs16 ---
Mode: from_scratch, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5156 - loss: 0.6945
Epoch 1: val_loss improved from inf to 0.62500, saving model to results\VGG19_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.5156 - loss: 0.6945 - val_accuracy: 0.6371 - val_loss: 0.6250 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5495 - loss: 0.6900
Epoch 2: val_loss improved from 0.62500 to 0.58716, saving model to results\VGG19_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 110s 2s/step - accuracy: 0.5500 - loss: 0.6896 - val_accuracy: 0.6532 - val_loss: 0.5872 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/s

In [9]:
run_experiment(
    model_choice="VGG19",
    training_mode='transfer_learning'
)

--- Starting Experiment: VGG19_transfer_learning_bs16 ---
Mode: transfer_learning, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 557ms/step - accuracy: 0.5122 - loss: 1.0450 - val_accuracy: 0.5887 - val_loss: 0.6358
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 573ms/step - accuracy: 0.4909 - loss: 1.0130 - val_accuracy: 0.6855 - val_loss: 0.6223
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 573ms/step - accuracy: 0.5196 - loss: 0.8717 - val_accuracy: 0.7419 - val_loss: 0.6193
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 572ms/step - accuracy: 0.5268 - loss: 0.8612 - val_accuracy: 0.7742 - val_loss: 0.6158
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 573ms/step - accuracy: 0.5066 - loss: 0.8676 - val_accuracy: 0.7984 - val_loss: 0.6093
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 

In [19]:
run_experiment(
    model_choice="VGG19",
    training_mode='fine_tune'
)

--- Starting Experiment: VGG19_fine_tune_bs16 ---
Mode: fine_tune, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 38s 573ms/step - accuracy: 0.4858 - loss: 0.7496 - val_accuracy: 0.6613 - val_loss: 0.6760
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 578ms/step - accuracy: 0.5456 - loss: 0.7091 - val_accuracy: 0.6855 - val_loss: 0.6695
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 580ms/step - accuracy: 0.5429 - loss: 0.7235 - val_accuracy: 0.7097 - val_loss: 0.6575
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 582ms/step - accuracy: 0.5679 - loss: 0.7007 - val_accuracy: 0.7097 - val_loss: 0.6417
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 577ms/step - accuracy: 0.5513 - loss: 0.6996 - val_accuracy: 0.7177 - val_loss: 0.6317
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 580ms/step - accuracy: 0.5840 - loss: 0.6795 - val_accuracy: 0.7258 - v

In [8]:
run_experiment(
    model_choice="VGG19",
    training_mode='full_monty'
)

--- Starting Experiment: VGG19_full_monty_bs16 ---
Mode: full_monty, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - accuracy: 0.4825 - loss: 0.9374
Epoch 1: val_loss improved from inf to 0.78393, saving model to results\VGG19_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 38s 571ms/step - accuracy: 0.4828 - loss: 0.9366 - val_accuracy: 0.4113 - val_loss: 0.7839 - learning_rate: 1.0000e-04
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 504ms/step - accuracy: 0.5292 - loss: 0.7591
Epoch 2: val_loss improved from 0.78393 to 0.71811, saving model to results\VGG19_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 37s 572ms/step - accuracy: 0.5292 - loss: 0.7592 - val_accuracy: 0.4113 - val_loss: 0.7181 - learning_rate: 1.0000e-04
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 508ms/step - acc

In [10]:
run_experiment(
    model_choice="ResNet50",
    training_mode='from_scratch'
)

--- Starting Experiment: ResNet50_from_scratch_bs16 ---
Mode: from_scratch, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6296 - loss: 1.0094
Epoch 1: val_loss improved from inf to 2.16371, saving model to results\ResNet50_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 129s 2s/step - accuracy: 0.6298 - loss: 1.0073 - val_accuracy: 0.5887 - val_loss: 2.1637 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7168 - loss: 0.7694
Epoch 2: val_loss did not improve from 2.16371
64/64 ━━━━━━━━━━━━━━━━━━━━ 96s 2s/step - accuracy: 0.7169 - loss: 0.7697 - val_accuracy: 0.5887 - val_loss: 4.2559 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7162 - loss: 0.7602
Epoch 3: val_loss did not i

In [11]:
run_experiment(
    model_choice="ResNet50",
    training_mode='transfer_learning'
)

--- Starting Experiment: ResNet50_transfer_learning_bs16 ---
Mode: transfer_learning, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 36s 474ms/step - accuracy: 0.4777 - loss: 0.8856 - val_accuracy: 0.3871 - val_loss: 0.7150
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 438ms/step - accuracy: 0.4779 - loss: 0.8229 - val_accuracy: 0.3710 - val_loss: 0.7056
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 445ms/step - accuracy: 0.5106 - loss: 0.8092 - val_accuracy: 0.4032 - val_loss: 0.6996
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 29s 446ms/step - accuracy: 0.5135 - loss: 0.7781 - val_accuracy: 0.4113 - val_loss: 0.6996
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 436ms/step - accuracy: 0.4960 - loss: 0.7722 - val_accuracy: 0.6210 - val_loss: 0.6870
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 438ms/step - accuracy: 0.5057 - loss: 0.7676

In [20]:
run_experiment(
    model_choice="ResNet50",
    training_mode='fine_tune'
)

--- Starting Experiment: ResNet50_fine_tune_bs16 ---
Mode: fine_tune, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 54s 770ms/step - accuracy: 0.5264 - loss: 0.9117 - val_accuracy: 0.6129 - val_loss: 0.6853
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 47s 737ms/step - accuracy: 0.5050 - loss: 0.8839 - val_accuracy: 0.5323 - val_loss: 0.6904
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 47s 737ms/step - accuracy: 0.4830 - loss: 0.8670 - val_accuracy: 0.4355 - val_loss: 0.6934
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 47s 733ms/step - accuracy: 0.4891 - loss: 0.8432 - val_accuracy: 0.5565 - val_loss: 0.6869
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 47s 734ms/step - accuracy: 0.4991 - loss: 0.8232 - val_accuracy: 0.6855 - val_loss: 0.6785
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 47s 737ms/step - accuracy: 0.4793 - loss: 0.8090 - val_accuracy: 0.6935 

In [9]:
run_experiment(
    model_choice="ResNet50",
    training_mode='full_monty'
)

--- Starting Experiment: ResNet50_full_monty_bs16 ---
Mode: full_monty, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 391ms/step - accuracy: 0.4918 - loss: 0.8969
Epoch 1: val_loss improved from inf to 0.69322, saving model to results\ResNet50_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 35s 475ms/step - accuracy: 0.4919 - loss: 0.8966 - val_accuracy: 0.4516 - val_loss: 0.6932 - learning_rate: 1.0000e-04
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step - accuracy: 0.4883 - loss: 0.9190
Epoch 2: val_loss improved from 0.69322 to 0.68368, saving model to results\ResNet50_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 28s 442ms/step - accuracy: 0.4884 - loss: 0.9185 - val_accuracy: 0.5968 - val_loss: 0.6837 - learning_rate: 1.0000e-04
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 381ms/s

In [12]:
run_experiment(
    model_choice="ResNet152",
    training_mode='from_scratch'
)

--- Starting Experiment: ResNet152_from_scratch_bs16 ---
Mode: from_scratch, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Training from Scratch ---
Epoch 1/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6169 - loss: 1.0801
Epoch 1: val_loss improved from inf to 1.96719, saving model to results\ResNet152_from_scratch_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 358s 4s/step - accuracy: 0.6170 - loss: 1.0792 - val_accuracy: 0.5887 - val_loss: 1.9672 - learning_rate: 1.0000e-04
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.6008 - loss: 1.0304
Epoch 2: val_loss did not improve from 1.96719
64/64 ━━━━━━━━━━━━━━━━━━━━ 259s 4s/step - accuracy: 0.6009 - loss: 1.0292 - val_accuracy: 0.5887 - val_loss: 5.2653 - learning_rate: 1.0000e-04
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.7174 - loss: 0.7239
Epoch 3: val_loss did no

In [13]:
run_experiment(
    model_choice="ResNet152",
    training_mode='transfer_learning'
)

--- Starting Experiment: ResNet152_transfer_learning_bs16 ---
Mode: transfer_learning, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Transfer Learning ---
234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.5044 - loss: 0.9298 - val_accuracy: 0.5726 - val_loss: 0.6742
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.4665 - loss: 0.8445 - val_accuracy: 0.6129 - val_loss: 0.6799
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.5497 - loss: 0.7744 - val_accuracy: 0.5968 - val_loss: 0.6891
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.5289 - loss: 0.7698 - val_accuracy: 0.6210 - val_loss: 0.6833
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.4657 - loss: 0.7963 - val_accuracy: 0.6048 - val_loss: 0.6873
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/s

In [21]:
run_experiment(
    model_choice="ResNet152",
    training_mode='fine_tune'
)

--- Starting Experiment: ResNet152_fine_tune_bs16 ---
Mode: fine_tune, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Fine Tune ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 153s 2s/step - accuracy: 0.5075 - loss: 1.3522 - val_accuracy: 0.5887 - val_loss: 0.7087
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5266 - loss: 0.9017 - val_accuracy: 0.5887 - val_loss: 0.6713
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5304 - loss: 0.8623 - val_accuracy: 0.4274 - val_loss: 0.6917
Epoch 4/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5226 - loss: 0.8527 - val_accuracy: 0.5968 - val_loss: 0.6849
Epoch 5/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.4901 - loss: 0.8190 - val_accuracy: 0.6694 - val_loss: 0.6817
Epoch 6/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.5017 - loss: 0.8065 - val_accuracy: 0.5887 - val_loss:

In [10]:
run_experiment(
    model_choice="ResNet152",
    training_mode='full_monty'
)

--- Starting Experiment: ResNet152_full_monty_bs16 ---
Mode: full_monty, Batch Size: 16, LR: 0.0001, Input: 224x224

Loading data from 'processed_data\BrinjalFruitX_224x224_balanced_classless'...
Data loaded successfully.

--- CONFIGURATION: Full Monty ---
Epoch 1/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5131 - loss: 0.8203
Epoch 1: val_loss improved from inf to 0.69498, saving model to results\ResNet152_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 105s 1s/step - accuracy: 0.5131 - loss: 0.8201 - val_accuracy: 0.5565 - val_loss: 0.6950 - learning_rate: 1.0000e-04
Epoch 2/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5061 - loss: 0.8359
Epoch 2: val_loss improved from 0.69498 to 0.68687, saving model to results\ResNet152_full_monty_bs16\best_model.keras
64/64 ━━━━━━━━━━━━━━━━━━━━ 83s 1s/step - accuracy: 0.5062 - loss: 0.8355 - val_accuracy: 0.5887 - val_loss: 0.6869 - learning_rate: 1.0000e-04
Epoch 3/50
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accur